In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re
import time

In [2]:
df_seinen_big_publisher = pd.read_csv('seinen_big_publisher.csv')

In [3]:
df_seinen_big_publisher

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,ID,Title,Type,Number of Scanlation group,Awarded,Japanese publihsers,...,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Smut,Sports,Supernatural,Tragedy,Yaoi,Yuri
0,22,22,22,0.0,1231,1 Pound no Fukuin,Manga,3,False,Shogakukan,...,0,0,0,1,0,1,0,0,0,0
1,27,2,0,NaN,72678,10 Dance,Manga,2,False,"Kodansha, Takeshobo",...,0,0,1,1,0,1,0,0,0,0
2,43,18,0,NaN,145740,1122,Manga,0,False,Kodansha,...,0,0,0,1,0,0,0,0,0,0
3,70,21,0,NaN,319,20 Seiki Shounen,Manga,2,True,Shogakukan,...,0,0,0,0,0,0,0,1,0,0
4,73,24,0,NaN,5809,2001 Ya Monogatari,Manga,0,False,"Chuuoukouronsha, Futabasha, Kobunsha, Shimanak...",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,9552,20,0,NaN,111876,Tomodachi x Monster,Manga,0,False,Futabasha,...,0,0,0,0,0,0,0,0,0,0
443,9556,24,0,NaN,163100,Tonari no Helvetica: Manga de Wakaru Oubun Fon...,Manga,0,False,Film Art-sha,...,0,0,0,1,0,0,0,0,0,0
444,9579,22,0,NaN,138820,Tongari Boushi no Atelier,Manga,5,True,Kodansha,...,0,0,0,0,0,0,0,0,0,0
445,9589,7,0,NaN,133631,Toppu GP,Manga,0,False,Kodansha,...,0,0,0,0,0,1,0,0,0,0


In [16]:
s = requests.Session()
s.proxies = {}
s.proxies['http'] = 'socks5h://localhost:9150'
s.proxies['https'] = 'socks5h://localhost:9150'
headers = {}
headers['User-agent'] = "HotJava/1.1.2 FCS"
s.cookies.clear()

In [17]:
def myGet(url):
    for i in range(10):
        try:
            time.sleep(1)
            req = s.get(url, verify=True)
            break
        except:
            continue
    else:
        print("Failed 10 times, skip this" + url)
        req = None
    return req

In [19]:
link = 'https://www.mangaupdates.com/series.html?id=1231'
req = myGet(link)
d_soup = bs(req.content)

In [28]:
title_unclean = d_soup.find_all('div','sContent')[3]

In [48]:
title_clean = str(title_unclean).replace('<div class="sContent">', '')

In [49]:
title_clean = title_clean.replace('<br/>\n</div>','')

In [50]:
title_clean

'1ポンドの福音<br/>1磅的福音<br/>1파운드의 복음<br/>Ichi Pound no Fukuin<br/>One Pound Gospel<br/>One-Pound Gospel<br/>The One Pound Gospel<br/>一磅的福音'

In [51]:
all_title = title_clean.split('<br/>')

In [52]:
all_title

['1ポンドの福音',
 '1磅的福音',
 '1파운드의 복음',
 'Ichi Pound no Fukuin',
 'One Pound Gospel',
 'One-Pound Gospel',
 'The One Pound Gospel',
 '一磅的福音']

In [61]:
artist_name = d_soup.find_all('div','sContent')[19].text.strip()

In [64]:
artist_name = artist_name.lower()

In [65]:
artist_name

'takahashi rumiko'

In [116]:
link_viz = 'https://www.viz.com/search?search={}'.format(all_title[4])

In [117]:
req_viz = myGet(link_viz)
soup_viz = bs(req_viz.content)

In [118]:
soup_viz.find_all("h3","section_title")[0]

<h3 class="section_title type-md type-lg--lg line-solid float-l--lg">1 Series</h3>

In [137]:
title = soup_viz.find_all("a","pos-r disp-bl bg-off-black color-white hover-bg-red o_property-link")[0].get('href')

In [138]:
title

'/one-pound-gospel'

In [140]:
link_title = 'https://www.viz.com'+title

In [141]:
req_viz_title = myGet(link_title)
soup_viz_title = bs(req_viz_title.content)

In [144]:
title_vol1 = soup_viz_title.find_all("a", "product-thumb ar-inner type-center")[0]

In [149]:
title_vol1_link_part = title_vol1.get('href')


In [150]:
title_vol1_link = 'https://www.viz.com'+title_vol1_link_part

In [151]:
req_vol1 = myGet(title_vol1_link)
soup_vol1 = bs(req_vol1.content)

In [155]:
soup_vol1.find_all("div","o_release-date mar-b-md")[0].text

'Release June 10, 2008'

In [160]:
pd.set_option('display.max_columns', None)


# GET VIZ'S TITLES PUBLISHING DATE

In [177]:
#Get the soup of the series
def get_soup(id):
    link = 'https://www.mangaupdates.com/series.html?id={}'.format(id)
    req = myGet(link)
    d_soup = bs(req.content) 
    return d_soup

In [165]:
#Get the alternative name of the series
def get_title(d_soup):
    title_unclean = d_soup.find_all('div','sContent')[3]
    title_clean = str(title_unclean).replace('<div class="sContent">', '')
    title_clean = title_clean.replace('<br/>\n</div>','')
    all_title = title_clean.split('<br/>')
    return all_title
    

In [189]:
def correct_name(title):
    link_viz = 'https://www.viz.com/search?search={}'.format(title) # search
    req_viz = myGet(link_viz) 
    soup_viz = bs(req_viz.content)
    search_success = soup_viz.find_all("h3","section_title")
    if search_success != []:
        return soup_viz
    else:
        return False

In [236]:
def get_date(soup_viz,title):
    title = soup_viz.find_all("a","pos-r disp-bl bg-off-black color-white hover-bg-red o_property-link")[0].get('href')
    link_title = 'https://www.viz.com'+title # go to the successful search link
    req_viz_title = myGet(link_title)
    soup_viz_title = bs(req_viz_title.content)
    title_vol1 = soup_viz_title.find_all("a", "product-thumb ar-inner type-center")[0]
    title_vol1_link_part = title_vol1.get('href')
    title_vol1_link = 'https://www.viz.com'+title_vol1_link_part #go to link of vol 1
    req_vol1 = myGet(title_vol1_link)
    soup_vol1 = bs(req_vol1.content)
    date = soup_vol1.find_all("div","o_release-date mar-b-md")[0].text
    return date[-4:]
        
        
        

In [171]:
viz_seinen = df_seinen_big_publisher[df_seinen_big_publisher['name of English publisher'].str.contains("Viz")]

In [173]:
viz_seinen_id = []

In [174]:
for id in viz_seinen['ID']:
    viz_seinen_id.append(id)

In [175]:
viz_seinen_id

[1231,
 319,
 5809,
 5673,
 2629,
 2249,
 119928,
 105948,
 788,
 151117,
 3445,
 156776,
 1409,
 2250,
 61649,
 6009,
 888,
 1447,
 227,
 110801,
 1145,
 3558,
 5195,
 1380,
 209,
 11060,
 7250,
 7410,
 2695,
 58623,
 109273,
 13815,
 19686,
 9963,
 147413,
 113608,
 115383,
 2157,
 5478,
 2647,
 3946,
 28126,
 10585,
 4006,
 5950,
 126648,
 153050,
 6385,
 162,
 8969,
 121872,
 7511,
 1437,
 26571,
 113410,
 2290,
 74899,
 16705,
 4630,
 62451,
 13388,
 2156,
 157931,
 6812,
 90524,
 120731,
 133910,
 2406,
 1051,
 18021,
 6212,
 127,
 145550,
 81736,
 19604,
 144765,
 141404,
 114105,
 6811,
 1429,
 4116,
 5582,
 80345,
 19473,
 11346,
 6173,
 6213,
 292,
 6671,
 1317,
 339,
 6017,
 39704,
 142,
 139808,
 11941,
 127539,
 142083,
 5490,
 1816,
 1786,
 8987,
 35172,
 5492,
 1427,
 79,
 11761,
 3435,
 2833,
 2272,
 5836,
 60566,
 4488,
 4115,
 372,
 62348,
 28795,
 7211,
 115995]

In [228]:
title_test = get_title(get_soup(viz_seinen_id[3]))

In [237]:
count = 0
for i in title_test: 
    if correct_name(i) != False:
        count += 1
        print(get_date(correct_name(i),i))
        break
if count == 0:
    print("find it yourself")

2018


In [238]:
dict_seinen = {}

In [242]:
import datetime
start_find_date = datetime.datetime.now()
for i in range(0, 10): 
    title_list = get_title(get_soup(viz_seinen_id[i]))
    count = 0
    for name in title_list: 
        if correct_name(name) != False:
            count +=1
            dict_seinen[viz_seinen_id[i]] = get_date(correct_name(name),name)
        if count == 0:
            dict_seinen[viz_seinen_id[i]] = "find it yourself"
    time_process = datetime.datetime.now() - start_find_date
    print(time_process)

0:00:24.286470
0:00:55.410344
0:01:06.791106
0:01:19.351711
0:01:32.052692
0:01:36.752835
0:01:42.082332
0:01:58.310841
0:02:24.571222
0:02:40.647732


In [243]:
dict_seinen

{1231: '2008',
 319: '2018',
 5809: 'find it yourself',
 5673: '2018',
 2629: '2018',
 2249: 'find it yourself',
 119928: 'find it yourself',
 105948: 'find it yourself',
 788: '2005',
 151117: '2006'}

In [221]:
title_test

['2001 Nights',
 '2001 Noches',
 '2001 ночь во Вселенной',
 '2001 밤 이야기',
 '2001-ya Monogatari',
 '2001夜物語',
 '2001夜物语',
 'Nisenichi Ya Monogatari']

In [199]:
title_test

['20 Seiki Shōnen',
 '20. Yüzyıl Çocukları',
 '20th Century Boys',
 '20th Século Meninos',
 '20thCB',
 '20世紀少年',
 '20세기 소년',
 'Abad ke-20 Anak Laki-laki',
 'amigo símbolo chicos del siglo XX',
 'Chłopaki z dwudziestego wieku',
 'Garçons du 20ème siècle',
 'I ragazzi del XX secolo',
 'Jungen des 20. Jahrhunderts',
 'Những Chàng Trai Thế Kỉ 20',
 'Nijuu Seiki Shounen',
 'Twentieth Century Boys',
 'Мальчишки двадцатого века',
 'فتيان القرن العشرين']

In [185]:
viz_seinen

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,ID,Title,Type,Number of Scanlation group,Awarded,Japanese publihsers,Oneshot,Scan start day,Scan latest day,Digitally published first by publisher,Simulus translate,Publish status,Completely scanlated,Artist,Publish year,Licensed,name of English publisher,English publishing status,Anime,Rating,Vote number,Action,Adult,Adventure,Comedy,Doujinshi,Drama,Ecchi,Fantasy,Gender Bender,Harem,Hentai,Historical,Horror,Josei,Lolicon,Martial Arts,Mature,Mecha,Mystery,Psychological,Romance,School Life,Sci-fi,Seinen,Shotacon,Shoujo,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Smut,Sports,Supernatural,Tragedy,Yaoi,Yuri
0,22,22,22,0.0,1231,1 Pound no Fukuin,Manga,3,False,Shogakukan,False,01/17/07,06/14/09,False,False,Complete,Yes,TAKAHASHI Rumiko,1989.0,Yes,Viz,4 Vols - Complete,False,7.2,94,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0
3,70,21,0,NaN,319,20 Seiki Shounen,Manga,2,True,Shogakukan,False,03/22/05,08/14/07,False,False,Complete,Yes,URASAWA Naoki,1999.0,Yes,Viz,22 Vols - Complete / 8 Perfect Eds - Ongoing,False,8.8,1548,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0
4,73,24,0,NaN,5809,2001 Ya Monogatari,Manga,0,False,"Chuuoukouronsha, Futabasha, Kobunsha, Shimanak...",False,Pass,Pass,False,False,Complete,Yes,HOSHINO Yukinobu,1984.0,Yes,Viz,3 Vols - Complete,False,8.1,85,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
5,77,3,0,NaN,5673,21 Seiki Shounen,Manga,3,True,Shogakukan,False,06/11/07,02/14/08,False,False,Complete,Yes,URASAWA Naoki,2006.0,Yes,Viz,2 Vols - Complete,False,8.5,948,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0
10,208,9,0,NaN,2629,Abara,Manga,3,False,Shueisha,False,01/04/06,12/25/07,False,False,Complete,Yes,NIHEI Tsutomu,2006.0,Yes,Viz,1 Vol - Complete,False,6.9,435,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,9075,19,0,NaN,372,Tenjou Tenge,Manga,5,False,Shueisha,False,09/08/06,04/27/11,False,False,Complete,Yes,Oh! Great,1997.0,Yes,"CMX, Viz","Defunct / 18 Vols, 11 Vols - Complete",True,7.2,905,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0
434,9120,14,0,NaN,62348,Terra Formars,Manga,7,True,Shueisha,False,06/03/12,Pass,False,True,Unknown,No,TACHIBANA Kenichi,2011.0,Yes,"MANGA Plus, Viz",22 Vols - Hiatus,True,7.4,344,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
435,9121,15,0,NaN,28795,Tesoro,Manga,1,False,Shogakukan,False,03/05/09,04/05/09,False,False,Complete,Yes,ONO Natsume,2008.0,Yes,Viz,Complete,False,7.1,19,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
437,9157,1,0,NaN,7211,The Big O,Manga,1,False,Kodansha,False,Unknown,Unknown,False,False,Complete,Yes,ARIGA Hitoshi,1999.0,Yes,Viz,6 Volumes - Complete,False,6.8,24,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0


In [235]:
string = "Release September 18, 2018"
string[-4:]

'2018'